<a href="https://colab.research.google.com/github/ryanpereirax/Intrusion-detector-with-supervised-models/blob/main/fuzzy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
fuzzy_df = pd.read_csv('/content/Fuzzy_dataset.csv')


In [ ]:
# Amostras de 1% do Dataset Original
amostras_R = fuzzy_df[fuzzy_df['R'] == 'R'].sample(n=5_000, random_state=42)
amostras_T = fuzzy_df[fuzzy_df['R'] == 'T'].sample(n=4_000, random_state=42)
fuzzy_df_balanceado = pd.concat([amostras_R, amostras_T]).sample(frac=1, random_state=42).reset_index(drop=True)

fuzzy_df_balanceado.to_csv('fuzzy_df_proporcional_ajustado.csv', index=False)

In [ ]:
pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 5.2 MB/s eta 0:00:00


In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Carregar o arquivo para examinar os dados
file_path = "/content/dos_df_proporcional_ajustado.csv"
data = pd.read_csv(file_path)

# Verificar as primeiras linhas do dataset para compreender sua estrutura
data.head()


,1478195721.903877,0545,8,d8,00,00.1,8a,00.2,00.3,00.4,00.5,R
0,1.478196e+09,05f4,8,29,17,cf,bf,be,3c,2a,dd,T
1,1.478198e+09,0130,8,0f,80,00,ff,22,80,06,e4,R
2,1.478198e+09,0316,8,05,22,7c,09,22,21,00,6f,R
3,1.478201e+09,04f0,8,00,00,00,80,00,69,d1,13,R
4,1.478198e+09,0359,8,ed,28,fa,f5,34,e3,17,bd,T


In [ ]:
# Renomear as colunas para melhor compreensão
data.columns = ['Timestamp', 'CAN_ID', 'DLC', 'DATA_0', 'DATA_1', 'DATA_2',
                'DATA_3', 'DATA_4', 'DATA_5', 'DATA_6', 'DATA_7', 'Flag']

# Verificar se existem valores ausentes
missing_data = data.isnull().sum()

# Analisar a proporção de flags 'T' e 'R'
flag_distribution = data['Flag'].value_counts(normalize=True)

# Examinar estatísticas descritivas para colunas numéricas
numeric_stats = data.describe()

missing_data, flag_distribution, numeric_stats


(Timestamp    0
 CAN_ID       0
 DLC          0
 DATA_0       0
 DATA_1       0
 DATA_2       0
 DATA_3       0
 DATA_4       0
 DATA_5       0
 DATA_6       0
 DATA_7       0
 Flag         0
 dtype: int64,
 Flag
 R    0.555556
 T    0.444444
 Name: proportion, dtype: float64,
           Timestamp     DLC
 count  9.000000e+03  9000.0
 mean   1.478197e+09     8.0
 std    1.624909e+03     0.0
 min    1.478196e+09     8.0
 25%    1.478196e+09     8.0
 50%    1.478197e+09     8.0
 75%    1.478198e+09     8.0
 max    1.478201e+09     8.0)

In [ ]:
# Converter o Timestamp para um formato de data legível para análise temporal
data['Timestamp'] = pd.to_datetime(data['Timestamp'], unit='s')

# Criar uma nova coluna para armazenar apenas a hora das mensagens
data['Hour'] = data['Timestamp'].dt.hour

# Contar o número de mensagens por hora
messages_per_hour = data.groupby('Hour').size()

# Identificar quais CAN_IDs são mais frequentes
can_id_distribution = data['CAN_ID'].value_counts().head(10)

messages_per_hour, can_id_distribution


(Hour
 17     873
 18    6736
 19    1391
 dtype: int64,
 CAN_ID
 043f    316
 0440    289
 0260    288
 02a0    285
 02c0    284
 0316    281
 0153    273
 0002    270
 0370    269
 0350    269
 Name: count, dtype: int64)

In [ ]:
# Verificar a distribuição dos CAN_IDs em relação às flags 'T' e 'R'
can_id_flag_distribution = data.groupby(['CAN_ID', 'Flag']).size().unstack(fill_value=0)

# Explorar padrões nos campos 'DATA_X' para mensagens com flag 'T'
data_injected = data[data['Flag'] == 'T']
data_normal = data[data['Flag'] == 'R']

# Calcular estatísticas descritivas para os campos 'DATA_X' para cada tipo de flag
data_injected_stats = data_injected[['DATA_0', 'DATA_1', 'DATA_2', 'DATA_3',
                                     'DATA_4', 'DATA_5', 'DATA_6', 'DATA_7']].describe()

data_normal_stats = data_normal[['DATA_0', 'DATA_1', 'DATA_2', 'DATA_3',
                                 'DATA_4', 'DATA_5', 'DATA_6', 'DATA_7']].describe()

can_id_flag_distribution, data_injected_stats, data_normal_stats


(Flag      R  T
 CAN_ID        
 0000      0  1
 0001      0  3
 0002    263  7
 0003      0  1
 0004      0  4
 ...     ... ..
 07fa      0  4
 07fb      0  2
 07fc      0  3
 07fe      0  1
 07ff      0  1
 
 [1752 rows x 2 columns],
        DATA_0 DATA_1 DATA_2 DATA_3 DATA_4 DATA_5 DATA_6 DATA_7
 count    4000   4000   4000   4000   4000   4000   4000   4000
 unique    256    256    256    256    256    256    256    256
 top        7e     5f     83     34     0e     46     ae     13
 freq       26     28     25     27     33     27     27     27,
        DATA_0 DATA_1 DATA_2 DATA_3 DATA_4 DATA_5 DATA_6 DATA_7
 count    5000   5000   5000   5000   5000   5000   5000   5000
 unique     69     49     50     26    120    139     59    251
 top        00     00     00     00     00     00     00     00
 freq     1714   2044   2865   2216   2195   1609   2677   2594)

In [ ]:
# Verificar a distribuição temporal das mensagens com CAN_ID '0000' (flag 'T')
injected_id_0000 = data[(data['CAN_ID'] == '0000') & (data['Flag'] == 'T')]

# Contagem de mensagens injetadas por minuto
injected_id_0000['Minute'] = injected_id_0000['Timestamp'].dt.minute
injected_per_minute = injected_id_0000.groupby('Minute').size()

# Explorar CAN_IDs que aparecem menos frequentemente para identificar possíveis outliers
rare_can_ids = data['CAN_ID'].value_counts().tail(10)

# Verificar se existem correlações entre os campos 'DATA_X' para as mensagens normais
data_normal_numeric = data_normal[['DATA_0', 'DATA_1', 'DATA_2', 'DATA_3',
                                   'DATA_4', 'DATA_5', 'DATA_6', 'DATA_7']].apply(lambda x: x.apply(int, base=16))
correlation_matrix = data_normal_numeric.corr()

injected_per_minute, rare_can_ids, correlation_matrix


<ipython-input-16-8fa582d780b2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  injected_id_0000['Minute'] = injected_id_0000['Timestamp'].dt.minute


(Minute
 7    1
 dtype: int64,
 CAN_ID
 0637    1
 0365    1
 0794    1
 012b    1
 04c8    1
 0730    1
 05ed    1
 0274    1
 074a    1
 01a3    1
 Name: count, dtype: int64,
           DATA_0    DATA_1    DATA_2    DATA_3    DATA_4    DATA_5    DATA_6  \
 DATA_0  1.000000  0.173661 -0.180856 -0.207755  0.266923  0.099619 -0.134284   
 DATA_1  0.173661  1.000000  0.165328  0.195718 -0.086909  0.185022 -0.273766   
 DATA_2 -0.180856  0.165328  1.000000  0.091638  0.317471 -0.122777  0.269925   
 DATA_3 -0.207755  0.195718  0.091638  1.000000  0.087645  0.582435 -0.038304   
 DATA_4  0.266923 -0.086909  0.317471  0.087645  1.000000  0.333615  0.140847   
 DATA_5  0.099619  0.185022 -0.122777  0.582435  0.333615  1.000000 -0.007733   
 DATA_6 -0.134284 -0.273766  0.269925 -0.038304  0.140847 -0.007733  1.000000   
 DATA_7 -0.234620  0.136150  0.100103 -0.043233 -0.069945 -0.155275 -0.142880   
 
           DATA_7  
 DATA_0 -0.234620  
 DATA_1  0.136150  
 DATA_2  0.100103  
 DATA_3 -0.0

In [ ]:
# Converter os campos 'DATA_X' para valores numéricos para análise de correlação
data_numeric = data[['DATA_0', 'DATA_1', 'DATA_2', 'DATA_3',
                     'DATA_4', 'DATA_5', 'DATA_6', 'DATA_7']].apply(lambda x: x.apply(int, base=16))

# Adicionar a flag como numérica (T = 1, R = 0) para facilitar a análise
data_numeric['Flag'] = data['Flag'].apply(lambda x: 1 if x == 'T' else 0)

# Calcular a correlação entre os campos de dados e a flag
correlation_with_flag = data_numeric.corr()['Flag'].drop('Flag')

correlation_with_flag


,Flag
DATA_0,0.373055
DATA_1,0.548602
DATA_2,0.566564
DATA_3,0.334792
DATA_4,0.465301
DATA_5,0.338721
DATA_6,0.610562
DATA_7,0.490871


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, roc_auc_score, f1_score
)

# Supondo que 'data_numeric' esteja carregado corretamente
X = data_numeric.drop('Flag', axis=1)
y = data_numeric['Flag']

# Dividindo os dados em treino (60%), validação (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicando o MinMaxScaler nos conjuntos de treino, validação e teste
scaler_minmax = MinMaxScaler()

X_train_scaled = scaler_minmax.fit_transform(X_train)
X_val_scaled = scaler_minmax.transform(X_val)
X_test_scaled = scaler_minmax.transform(X_test)

# Criando o modelo de Regressão Logística
model_minmax = LogisticRegression(
    penalty='l2',
    solver='lbfgs',
    max_iter=500,
    random_state=42
)

# Treinando o modelo com os dados de treino
model_minmax.fit(X_train_scaled, y_train)

# Função para calcular e exibir métricas
def print_metrics(y_true, y_pred, y_prob, dataset_name):
    print(f"\nMétricas para o conjunto de {dataset_name}:")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_true, y_prob):.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))


# Avaliando no conjunto de treino
y_train_pred = model_minmax.predict(X_train_scaled)
y_train_prob = model_minmax.predict_proba(X_train_scaled)[:, 1]
print_metrics(y_train, y_train_pred, y_train_prob, "Treino")


# Avaliando no conjunto de teste
y_test_pred = model_minmax.predict(X_test_scaled)
y_test_prob = model_minmax.predict_proba(X_test_scaled)[:, 1]
print_metrics(y_test, y_test_pred, y_test_prob, "Teste")

# Avaliando no conjunto de validação
y_val_pred = model_minmax.predict(X_val_scaled)
y_val_prob = model_minmax.predict_proba(X_val_scaled)[:, 1]
print_metrics(y_val, y_val_pred, y_val_prob, "Validação")



Métricas para o conjunto de Treino:
Acurácia: 0.9561
Precisão: 0.9749
Recall: 0.9263
F1-Score: 0.9500
AUC-ROC: 0.9871

Matriz de Confusão:
[[2912   58]
 [ 179 2251]]

Métricas para o conjunto de Teste:
Acurácia: 0.9594
Precisão: 0.9688
Recall: 0.9382
F1-Score: 0.9532
AUC-ROC: 0.9912

Matriz de Confusão:
[[983  24]
 [ 49 744]]

Métricas para o conjunto de Validação:
Acurácia: 0.9589
Precisão: 0.9693
Recall: 0.9344
F1-Score: 0.9515
AUC-ROC: 0.9901

Matriz de Confusão:
[[1000   23]
 [  51  726]]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, roc_auc_score, f1_score
)

# Supondo que 'data_numeric' esteja carregado corretamente
X = data_numeric.drop('Flag', axis=1)
y = data_numeric['Flag']

# Dividindo os dados em treino (60%), validação (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicando o MinMaxScaler nos conjuntos de treino, validação e teste
scaler_minmax = MinMaxScaler()

X_train_scaled = scaler_minmax.fit_transform(X_train)
X_val_scaled = scaler_minmax.transform(X_val)
X_test_scaled = scaler_minmax.transform(X_test)

# Criando e treinando o modelo Random Forest com regularização
model_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

model_rf.fit(X_train_scaled, y_train)

# Função para calcular e exibir métricas
def print_metrics_rf(y_true, y_pred, y_prob, dataset_name):
    print(f"\nMétricas para o conjunto de {dataset_name}:")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_true, y_prob):.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))


# Avaliar no conjunto de treino
y_train_pred_rf = model_rf.predict(X_train_scaled)
y_train_prob_rf = model_rf.predict_proba(X_train_scaled)[:, 1]
print_metrics_rf(y_train, y_train_pred_rf, y_train_prob_rf, "Treino")


# Avaliar no conjunto de teste
y_test_pred_rf = model_rf.predict(X_test_scaled)
y_test_prob_rf = model_rf.predict_proba(X_test_scaled)[:, 1]
print_metrics_rf(y_test, y_test_pred_rf, y_test_prob_rf, "Teste")

# Avaliar no conjunto de validação
y_val_pred_rf = model_rf.predict(X_val_scaled)
y_val_prob_rf = model_rf.predict_proba(X_val_scaled)[:, 1]
print_metrics_rf(y_val, y_val_pred_rf, y_val_prob_rf, "Validação")


Métricas para o conjunto de Treino:
Acurácia: 1.0000
Precisão: 1.0000
Recall: 1.0000
F1-Score: 1.0000
AUC-ROC: 1.0000

Matriz de Confusão:
[[2970    0]
 [   0 2430]]

Métricas para o conjunto de Teste:
Acurácia: 1.0000
Precisão: 1.0000
Recall: 1.0000
F1-Score: 1.0000
AUC-ROC: 1.0000

Matriz de Confusão:
[[1007    0]
 [   0  793]]

Métricas para o conjunto de Validação:
Acurácia: 1.0000
Precisão: 1.0000
Recall: 1.0000
F1-Score: 1.0000
AUC-ROC: 1.0000

Matriz de Confusão:
[[1023    0]
 [   0  777]]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, roc_auc_score, f1_score
)

# Supondo que 'data_numeric' esteja carregado corretamente
X = data_numeric.drop('Flag', axis=1)
y = data_numeric['Flag']

# Dividindo os dados em treino (60%), validação (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicando o MinMaxScaler nos conjuntos de treino, validação e teste
scaler_minmax = MinMaxScaler()

X_train_scaled = scaler_minmax.fit_transform(X_train)
X_val_scaled = scaler_minmax.transform(X_val)
X_test_scaled = scaler_minmax.transform(X_test)

# Criando e treinando o modelo de Árvore de Decisão com regularização
model_tree = DecisionTreeClassifier(
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

model_tree.fit(X_train_scaled, y_train)

# Função para calcular e exibir métricas
def print_metrics_tree(y_true, y_pred, y_prob, dataset_name):
    print(f"\nMétricas para o conjunto de {dataset_name}:")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_true, y_prob):.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))


# Avaliar no conjunto de treino
y_train_pred_tree = model_tree.predict(X_train_scaled)
y_train_prob_tree = model_tree.predict_proba(X_train_scaled)[:, 1]
print_metrics_tree(y_train, y_train_pred_tree, y_train_prob_tree, "Treino")


# Avaliar no conjunto de teste
y_test_pred_tree = model_tree.predict(X_test_scaled)
y_test_prob_tree = model_tree.predict_proba(X_test_scaled)[:, 1]
print_metrics_tree(y_test, y_test_pred_tree, y_test_prob_tree, "Teste")

# Avaliar no conjunto de validação
y_val_pred_tree = model_tree.predict(X_val_scaled)
y_val_prob_tree = model_tree.predict_proba(X_val_scaled)[:, 1]
print_metrics_tree(y_val, y_val_pred_tree, y_val_prob_tree, "Validação")


Métricas para o conjunto de Treino:
Acurácia: 0.9978
Precisão: 1.0000
Recall: 0.9951
F1-Score: 0.9975
AUC-ROC: 0.9996

Matriz de Confusão:
[[2970    0]
 [  12 2418]]

Métricas para o conjunto de Teste:
Acurácia: 0.9972
Precisão: 0.9987
Recall: 0.9950
F1-Score: 0.9968
AUC-ROC: 0.9976

Matriz de Confusão:
[[1006    1]
 [   4  789]]

Métricas para o conjunto de Validação:
Acurácia: 0.9950
Precisão: 1.0000
Recall: 0.9884
F1-Score: 0.9942
AUC-ROC: 0.9959

Matriz de Confusão:
[[1023    0]
 [   9  768]]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, roc_auc_score, f1_score
)

# Supondo que 'data_numeric' esteja carregado corretamente
X = data_numeric.drop('Flag', axis=1)
y = data_numeric['Flag']

# Dividindo os dados em treino (60%), validação (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicando o MinMaxScaler nos conjuntos de treino, validação e teste
scaler_minmax = MinMaxScaler()

X_train_scaled = scaler_minmax.fit_transform(X_train)
X_val_scaled = scaler_minmax.transform(X_val)
X_test_scaled = scaler_minmax.transform(X_test)

# Criando e treinando o modelo de Gradient Boosting
model_gb = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

model_gb.fit(X_train_scaled, y_train)

# Função para calcular e exibir métricas
def print_metrics_gb(y_true, y_pred, y_prob, dataset_name):
    print(f"\nMétricas para o conjunto de {dataset_name}:")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_true, y_prob):.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))

# Avaliar no conjunto de treino
y_train_pred_gb = model_gb.predict(X_train_scaled)
y_train_prob_gb = model_gb.predict_proba(X_train_scaled)[:, 1]
print_metrics_gb(y_train, y_train_pred_gb, y_train_prob_gb, "Treino")


# Avaliar no conjunto de teste
y_test_pred_gb = model_gb.predict(X_test_scaled)
y_test_prob_gb = model_gb.predict_proba(X_test_scaled)[:, 1]
print_metrics_gb(y_test, y_test_pred_gb, y_test_prob_gb, "Teste")

# Avaliar no conjunto de validação
y_val_pred_gb = model_gb.predict(X_val_scaled)
y_val_prob_gb = model_gb.predict_proba(X_val_scaled)[:, 1]
print_metrics_gb(y_val, y_val_pred_gb, y_val_prob_gb, "Validação")


Métricas para o conjunto de Treino:
Acurácia: 1.0000
Precisão: 1.0000
Recall: 1.0000
F1-Score: 1.0000
AUC-ROC: 1.0000

Matriz de Confusão:
[[2970    0]
 [   0 2430]]

Métricas para o conjunto de Teste:
Acurácia: 0.9983
Precisão: 0.9975
Recall: 0.9987
F1-Score: 0.9981
AUC-ROC: 0.9995

Matriz de Confusão:
[[1005    2]
 [   1  792]]

Métricas para o conjunto de Validação:
Acurácia: 0.9967
Precisão: 0.9974
Recall: 0.9949
F1-Score: 0.9961
AUC-ROC: 0.9976

Matriz de Confusão:
[[1021    2]
 [   4  773]]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, roc_auc_score, f1_score
)

# Supondo que 'data_numeric' esteja carregado corretamente
X = data_numeric.drop('Flag', axis=1)
y = data_numeric['Flag']

# Dividindo os dados em treino (60%), validação (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicando o MinMaxScaler nos conjuntos de treino, validação e teste
scaler_minmax = MinMaxScaler()

X_train_scaled = scaler_minmax.fit_transform(X_train)
X_val_scaled = scaler_minmax.transform(X_val)
X_test_scaled = scaler_minmax.transform(X_test)

# Criando e treinando o modelo K-Nearest Neighbors
model_knn = KNeighborsClassifier(
    n_neighbors=5,
    weights='uniform',
    metric='minkowski',
    n_jobs=-1
)

model_knn.fit(X_train_scaled, y_train)

# Função para calcular e exibir métricas
def print_metrics_knn(y_true, y_pred, y_prob, dataset_name):
    print(f"\nMétricas para o conjunto de {dataset_name}:")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_true, y_prob):.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))


# Avaliar no conjunto de treino
y_train_pred_knn = model_knn.predict(X_train_scaled)
y_train_prob_knn = model_knn.predict_proba(X_train_scaled)[:, 1]
print_metrics_knn(y_train, y_train_pred_knn, y_train_prob_knn, "Treino")


# Avaliar no conjunto de teste
y_test_pred_knn = model_knn.predict(X_test_scaled)
y_test_prob_knn = model_knn.predict_proba(X_test_scaled)[:, 1]
print_metrics_knn(y_test, y_test_pred_knn, y_test_prob_knn, "Teste")

# Avaliar no conjunto de validação
y_val_pred_knn = model_knn.predict(X_val_scaled)
y_val_prob_knn = model_knn.predict_proba(X_val_scaled)[:, 1]
print_metrics_knn(y_val, y_val_pred_knn, y_val_prob_knn, "Validação")



Métricas para o conjunto de Treino:
Acurácia: 0.9948
Precisão: 1.0000
Recall: 0.9885
F1-Score: 0.9942
AUC-ROC: 1.0000

Matriz de Confusão:
[[2970    0]
 [  28 2402]]

Métricas para o conjunto de Teste:
Acurácia: 0.9944
Precisão: 0.9987
Recall: 0.9887
F1-Score: 0.9937
AUC-ROC: 0.9970

Matriz de Confusão:
[[1006    1]
 [   9  784]]

Métricas para o conjunto de Validação:
Acurácia: 0.9928
Precisão: 1.0000
Recall: 0.9833
F1-Score: 0.9916
AUC-ROC: 0.9968

Matriz de Confusão:
[[1023    0]
 [  13  764]]


In [ ]:

from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, roc_auc_score, f1_score
)

# Supondo que 'data_numeric' esteja carregado corretamente
X = data_numeric.drop('Flag', axis=1)
y = data_numeric['Flag']

# Dividindo os dados em treino (60%), validação (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicando o MinMaxScaler nos conjuntos de treino, validação e teste
scaler_minmax = MinMaxScaler()

X_train_scaled = scaler_minmax.fit_transform(X_train)
X_val_scaled = scaler_minmax.transform(X_val)
X_test_scaled = scaler_minmax.transform(X_test)

# Criando e treinando o modelo SVM
model_svm = SVC(
    kernel='rbf',
    C=1.0,
    probability=True,
    random_state=42
)

model_svm.fit(X_train_scaled, y_train)

# Função para calcular e exibir métricas
def print_metrics_svm(y_true, y_pred, y_prob, dataset_name):
    print(f"\nMétricas para o conjunto de {dataset_name}:")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_true, y_prob):.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))

# Avaliar no conjunto de treino
y_train_pred_svm = model_svm.predict(X_train_scaled)
y_train_prob_svm = model_svm.predict_proba(X_train_scaled)[:, 1]
print_metrics_svm(y_train, y_train_pred_svm, y_train_prob_svm, "Treino")


# Avaliar no conjunto de teste
y_test_pred_svm = model_svm.predict(X_test_scaled)
y_test_prob_svm = model_svm.predict_proba(X_test_scaled)[:, 1]
print_metrics_svm(y_test, y_test_pred_svm, y_test_prob_svm, "Teste")

# Avaliar no conjunto de validação
y_val_pred_svm = model_svm.predict(X_val_scaled)
y_val_prob_svm = model_svm.predict_proba(X_val_scaled)[:, 1]
print_metrics_svm(y_val, y_val_pred_svm, y_val_prob_svm, "Validação")




Métricas para o conjunto de Treino:
Acurácia: 0.9965
Precisão: 1.0000
Recall: 0.9922
F1-Score: 0.9961
AUC-ROC: 0.9996

Matriz de Confusão:
[[2970    0]
 [  19 2411]]

Métricas para o conjunto de Teste:
Acurácia: 0.9983
Precisão: 1.0000
Recall: 0.9962
F1-Score: 0.9981
AUC-ROC: 1.0000

Matriz de Confusão:
[[1007    0]
 [   3  790]]

Métricas para o conjunto de Validação:
Acurácia: 0.9972
Precisão: 1.0000
Recall: 0.9936
F1-Score: 0.9968
AUC-ROC: 1.0000

Matriz de Confusão:
[[1023    0]
 [   5  772]]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, roc_auc_score, f1_score
)

X = data_numeric.drop('Flag', axis=1)
y = data_numeric['Flag']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler_minmax = MinMaxScaler()

X_train_scaled = scaler_minmax.fit_transform(X_train)
X_val_scaled = scaler_minmax.transform(X_val)
X_test_scaled = scaler_minmax.transform(X_test)

# Criando e treinando o modelo XGBoost
model_xgb = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=10,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

model_xgb.fit(X_train_scaled, y_train)

# Função para calcular e exibir métricas
def print_metrics_xgb(y_true, y_pred, y_prob, dataset_name):
    print(f"\nMétricas para o conjunto de {dataset_name}:")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_true, y_prob):.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))


# Avaliar no conjunto de treino
y_train_pred_xgb = model_xgb.predict(X_train_scaled)
y_train_prob_xgb = model_xgb.predict_proba(X_train_scaled)[:, 1]
print_metrics_xgb(y_train, y_train_pred_xgb, y_train_prob_xgb, "Treino")


# Avaliar no conjunto de teste
y_test_pred_xgb = model_xgb.predict(X_test_scaled)
y_test_prob_xgb = model_xgb.predict_proba(X_test_scaled)[:, 1]
print_metrics_xgb(y_test, y_test_pred_xgb, y_test_prob_xgb, "Teste")

# Avaliar no conjunto de validação
y_val_pred_xgb = model_xgb.predict(X_val_scaled)
y_val_prob_xgb = model_xgb.predict_proba(X_val_scaled)[:, 1]
print_metrics_xgb(y_val, y_val_pred_xgb, y_val_prob_xgb, "Validação")


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:36:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Métricas para o conjunto de Treino:
Acurácia: 1.0000
Precisão: 1.0000
Recall: 1.0000
F1-Score: 1.0000
AUC-ROC: 1.0000

Matriz de Confusão:
[[2970    0]
 [   0 2430]]

Métricas para o conjunto de Teste:
Acurácia: 1.0000
Precisão: 1.0000
Recall: 1.0000
F1-Score: 1.0000
AUC-ROC: 1.0000

Matriz de Confusão:
[[1007    0]
 [   0  793]]

Métricas para o conjunto de Validação:
Acurácia: 0.9994
Precisão: 1.0000
Recall: 0.9987
F1-Score: 0.9994
AUC-ROC: 1.0000

Matriz de Confusão:
[[1023    0]
 [   1  776]]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, roc_auc_score, f1_score
)

# Supondo que 'data_numeric' esteja carregado corretamente
X = data_numeric.drop('Flag', axis=1)
y = data_numeric['Flag']

# Dividindo os dados em treino (60%), validação (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicando o MinMaxScaler nos conjuntos de treino, validação e teste
scaler_minmax = MinMaxScaler()

X_train_scaled = scaler_minmax.fit_transform(X_train)
X_val_scaled = scaler_minmax.transform(X_val)
X_test_scaled = scaler_minmax.transform(X_test)

# Criando e treinando o modelo CatBoost
model_cb = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=10,
    random_seed=42,
    verbose=0
)

model_cb.fit(X_train_scaled, y_train)

# Função para calcular e exibir métricas
def print_metrics_cb(y_true, y_pred, y_prob, dataset_name):
    print(f"\nMétricas para o conjunto de {dataset_name}:")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_true, y_prob):.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))


# Avaliar no conjunto de treino
y_train_pred_cb = model_cb.predict(X_train_scaled)
y_train_prob_cb = model_cb.predict_proba(X_train_scaled)[:, 1]
print_metrics_cb(y_train, y_train_pred_cb, y_train_prob_cb, "Treino")


# Avaliar no conjunto de teste
y_test_pred_cb = model_cb.predict(X_test_scaled)
y_test_prob_cb = model_cb.predict_proba(X_test_scaled)[:, 1]
print_metrics_cb(y_test, y_test_pred_cb, y_test_prob_cb, "Teste")

# Avaliar no conjunto de validação
y_val_pred_cb = model_cb.predict(X_val_scaled)
y_val_prob_cb = model_cb.predict_proba(X_val_scaled)[:, 1]
print_metrics_cb(y_val, y_val_pred_cb, y_val_prob_cb, "Validação")



Métricas para o conjunto de Treino:
Acurácia: 1.0000
Precisão: 1.0000
Recall: 1.0000
F1-Score: 1.0000
AUC-ROC: 1.0000

Matriz de Confusão:
[[2970    0]
 [   0 2430]]

Métricas para o conjunto de Teste:
Acurácia: 0.9989
Precisão: 1.0000
Recall: 0.9975
F1-Score: 0.9987
AUC-ROC: 1.0000

Matriz de Confusão:
[[1007    0]
 [   2  791]]

Métricas para o conjunto de Validação:
Acurácia: 0.9989
Precisão: 1.0000
Recall: 0.9974
F1-Score: 0.9987
AUC-ROC: 1.0000

Matriz de Confusão:
[[1023    0]
 [   2  775]]
